## 准备数据

In [61]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [62]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [63]:
class myModel:
    def __init__(self):
        # 定义模型参数
        self.W1 = tf.Variable(tf.random.normal([784, 100]))  # 第一层权重，从784个特征映射到100个隐藏层神经元
        self.b1 = tf.Variable(tf.zeros([100]))  # 第一层偏置
        self.W2 = tf.Variable(tf.random.normal([100, 10]))  # 第二层权重，从100个隐藏层神经元映射到10个输出类别
        self.b2 = tf.Variable(tf.zeros([10]))  # 第二层偏置

    def __call__(self, x):
        # 实现模型前向传播
        x = tf.reshape(x, [-1, 784])  # 显式展平输入
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # 第一层隐藏层，使用 ReLU 激活函数
        logits = tf.matmul(h1, self.W2) + self.b2  # 第二层输出层
        return logits  # 返回未归一化的logits
    
    
model = myModel()
optimizer = optimizers.Adam()



## 计算 loss

In [64]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [65]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 88.81725 ; accuracy 0.083516665
epoch 1 : loss 84.14179 ; accuracy 0.08925
epoch 2 : loss 80.1928 ; accuracy 0.09538333
epoch 3 : loss 76.82547 ; accuracy 0.1027
epoch 4 : loss 73.90951 ; accuracy 0.1091
epoch 5 : loss 71.34558 ; accuracy 0.116166666
epoch 6 : loss 69.05755 ; accuracy 0.12206667
epoch 7 : loss 66.98651 ; accuracy 0.12825
epoch 8 : loss 65.089 ; accuracy 0.13416667
epoch 9 : loss 63.333282 ; accuracy 0.13926667
epoch 10 : loss 61.70271 ; accuracy 0.14481667
epoch 11 : loss 60.18046 ; accuracy 0.15028334
epoch 12 : loss 58.752716 ; accuracy 0.15585
epoch 13 : loss 57.412476 ; accuracy 0.16046667
epoch 14 : loss 56.153732 ; accuracy 0.16528334
epoch 15 : loss 54.96992 ; accuracy 0.1695
epoch 16 : loss 53.856415 ; accuracy 0.1743
epoch 17 : loss 52.807133 ; accuracy 0.17886667
epoch 18 : loss 51.814312 ; accuracy 0.18316667
epoch 19 : loss 50.8726 ; accuracy 0.18726666
epoch 20 : loss 49.977688 ; accuracy 0.1915
epoch 21 : loss 49.12324 ; accuracy 0.1957
epo